# Búsquedas aproximadas con Hierarchical K-means tree

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025 

Al igual que la semana anterior, usaremos la librería FLANN (Fast Library for Approximate Nearest Neighbors)
https://github.com/flann-lib/flann

Es una librería escrita en C++ que tiene un wrapper para python llamado PyFlann.

Instalar PyFlann `1.9.2` con:

```
# instalar con conda
conda install pyflann 
```

**Nota: NO usar pip** porque instalará una versión antigua `1.6.14`.

Si se está usando anaconda, instalar con: `conda install -c conda-forge pyflann`


### Gráficos interactivos

Para los gráficos se usa matplotlib:
```
pip install matplotlib
```

Para gráficos interactivos (por ej. hacer zoom):

  1. Instalar ipympl:  `pip install ipympl`
  2. Reiniciar jupyter 
  3. Reemplazar `%matplotlib inline` por `%matplotlib widget`


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Descomentar esta linea para graficos interactivos
## %matplotlib widget


## Funciones para evaluar

In [ ]:
# primero cargar algunas funciones auxiliares
import numpy
import time
import pyflann


# calcular la calidad de la respuesta contra el resultado "ideal" dado por el linear scan
class Experimento:
    def __init__(self, nombre, archivo_dataset_q, archivo_dataset_r):
        self.nombre = nombre
        # cargar conjuntos de vectores Q y R
        self.dataset_q = numpy.load(archivo_dataset_q)
        self.dataset_r = numpy.load(archivo_dataset_r)
        print(
            "{}: conjunto_Q={} conjunto_R={}".format(
                nombre, self.dataset_q.shape, self.dataset_r.shape
            )
        )

    def respuesta_ideal(self, nns, distancias, segundos):
        self.gt_nns = nns
        self.gt_distancias = distancias
        self.gt_tiempo = segundos
        self.correctas = 0
        self.incorrectas = 0
        self.tiempo = 0
        self.precision = 0
        self.eficiencia = 0
        print("{:<8}: Linear Scan toma {:.1f} segundos".format(self.nombre, segundos))

    def evaluar_busqueda(
        self, nombre_indice, nombre_busqueda, nns, distancias, segundos
    ):
        self.last_nombre_indice = nombre_indice
        self.last_nombre_busqueda = nombre_busqueda
        self.last_nns = nns
        self.last_distancias = distancias
        self.correctas = 0
        self.incorrectas = 0
        for i in range(len(self.gt_nns)):
            # es correcta si encontró el mismo elemento que el linear scan, o si
            # encontró otro elemento que está a la misma distancia
            if nns[i] == self.gt_nns[i] or distancias[i] == self.gt_distancias[i]:
                self.correctas += 1
            else:
                self.incorrectas += 1
        self.tiempo = segundos
        self.precision = self.correctas / (self.correctas + self.incorrectas)
        self.eficiencia = self.tiempo / self.gt_tiempo
        print(
            "{:<8}: {:<8} precision={:>6.1%}  tiempo={:>5.2f} seg. ({:>5.1%} del tiempo de Linear Scan)".format(
                nombre_indice,
                nombre_busqueda,
                self.precision,
                self.tiempo,
                self.eficiencia,
            )
        )
        return self.precision, self.eficiencia

    def mostrar_respuesta(self, id_query):
        numpy.set_printoptions(edgeitems=10, threshold=20, linewidth=200)
        vector_q = self.dataset_q[id_query]
        id_NN = self.last_nns[id_query]
        vector_NN = self.dataset_r[id_NN]
        dist_NN = self.last_distancias[id_query]
        dist_euclid = numpy.linalg.norm(vector_q - vector_NN)
        print()
        print("Vector q_{}: {}".format(id_query, vector_q))
        print(
            "{} {} encontró que el más cercano a q_{} es r_{}, que está a distancia={} (pyflann calcula distancia euclidiana^2)".format(
                self.last_nombre_indice,
                self.last_nombre_busqueda,
                id_query,
                id_NN,
                dist_NN,
            )
        )
        print("Vector r_{}: {}".format(id_NN, vector_NN))
        print(
            "distancia euclidiana real entre ambos vectores: {:.3f}".format(dist_euclid)
        )
        if (
            self.last_nns[id_query] == self.gt_nns[id_query]
            or self.last_distancias[id_query] == self.gt_distancias[id_query]
        ):
            print("CORRECTO")
        else:
            print(
                "INCORRECTO! (el más cercano es r_{} a distancia={})".format(
                    self.gt_nns[id_query], self.gt_distancias[id_query], dist_NN
                )
            )


experimentoA = Experimento("DATASET_A", "dataset_a_q.npy", "dataset_a_r.npy")

# Parte 1: Linear Scan como línea base

El resultado obtenido por linear scan (los vectores más cercanos) son usados como el modelo ideal para medir efectividad.

In [ ]:
# crear un objeto flann
flann = pyflann.FLANN()

# construir el indice para linear scan
flann.build_index(experimentoA.dataset_r, algorithm="linear")

# ejecutar el linear scan
t0 = time.time()

# se usa un solo core, debiera demorar unos 25 a 35 segundos
nns, dists = flann.nn_index(experimentoA.dataset_q, num_neighbors=1, cores=1)
segundos = time.time() - t0

# guardar las respuestas del linear scan
experimentoA.respuesta_ideal(nns, dists, segundos)

### Imprimir resultado del linear scan

In [ ]:
# solo para probar, se evaluará la misma búsqueda exacta (por definición obtendrá un 100%)
experimentoA.evaluar_busqueda("Linear Scan", "", nns, dists, segundos)

print(
    "respuestas_correctas  = {:4.0f} / {:4.0f}".format(
        experimentoA.correctas, experimentoA.correctas + experimentoA.incorrectas
    )
)
print(
    "respuestas_incorrectas= {:4.0f} / {:4.0f}".format(
        experimentoA.incorrectas, experimentoA.correctas + experimentoA.incorrectas
    )
)

### Revisar algunos resultados

Se muestran algunos resultados específicos. Notar que PyFlann entrega el cuadrado de la distancia euclidiana.

In [ ]:
experimentoA.mostrar_respuesta(0)
experimentoA.mostrar_respuesta(1)
experimentoA.mostrar_respuesta(2)

# Parte 2: Hierarchical K-means tree

Notar que la construcción y uso del Hierarchical K-means tree es bastante similar al k-d tree del ejemplo de la semana anterior, es necesario seleccionar `algorithm="kmeans"` y ajustar el parámetro ``branching`` (centroides de kmeans).

In [ ]:
# construir el indice kmeans tree
t0 = time.time()

# Documentación de FLANN en LaTeX:
#  https://github.com/mariusmuja/flann/tree/master/doc
flann.build_index(experimentoA.dataset_r, algorithm="kmeans", branching=5)

segundos_construccion = time.time() - t0

print("construcción k-means tree={:.2f} segundos".format(segundos_construccion))

### Evaluar la búsqueda aproximada

In [ ]:
# busqueda exacta del 1-NN usando el ultimo indice construido (kmeans tree)
t0 = time.time()

nns, dists = flann.nn_index(experimentoA.dataset_q, num_neighbors=1, cores=1, checks=5)
segundos = time.time() - t0

experimentoA.evaluar_busqueda(
    "kmeans tree 5-branching", "checks=5", nns, dists, segundos
)

print(
    "respuestas_correctas  = {:4.0f} / {:4.0f}".format(
        experimentoA.correctas, experimentoA.correctas + experimentoA.incorrectas
    )
)
print(
    "respuestas_incorrectas= {:4.0f} / {:4.0f}".format(
        experimentoA.incorrectas, experimentoA.correctas + experimentoA.incorrectas
    )
)

In [ ]:
experimentoA.mostrar_respuesta(0)
experimentoA.mostrar_respuesta(1)
experimentoA.mostrar_respuesta(2)

### Calcular curva de efectividad versus eficiencia

In [ ]:
precisiones = []
tiempos = []

# probar la búsqueda aproximada usando varios checks
for checks in (1, 10, 20, 40, 60, 80, 100, 200, 400, 600, 800, 1000, 2000, 4000):
    t0 = time.time()
    nns, dists = flann.nn_index(
        experimentoA.dataset_q, num_neighbors=1, cores=1, checks=checks
    )
    segundos = time.time() - t0

    precision, eficiencia = experimentoA.evaluar_busqueda(
        "kmeans tree 5-branching", "checks={:>4}".format(checks), nns, dists, segundos
    )

    precisiones.append(precision)
    tiempos.append(eficiencia)

### Mostrar curva de efectividad versus eficiencia

In [ ]:
plt.plot(precisiones, tiempos, label='kmeans tree', color='b', linestyle='-', marker='o', markerfacecolor='g', markersize=8)
plt.title(experimentoA.nombre)
plt.xlabel('Precisión NN [comparado con LS]')
plt.ylabel('Tiempos [comparado con LS]')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.show()

Al aumentar `checks` siempre aumenta el tiempo de búsqueda y mejora la precisión, sin embargo a tasas muy distintas como se ve en la forma de la curva del experimento anterior.

En el ejemplo anterior, al pasar de `checks=1` a `checks=10` el tiempo **aumenta poco** (pocas décimas de segundo), pero la precisión **mejora mucho** (desde unos 50% a como 75%).

Por otra parte, al pasar de `checks=1000` a `checks=2000` la precisión **mejora muy poco**, pero el tiempo **aumenta varios segundos**.

Para encontrar un buen valor para `checks` se necesita fijar un objetivo, ya sea en tiempo o en precisión

# Parte 3: Evaluar en otro dataset (Dataset_B)

In [ ]:
experimentoB = Experimento("DATASET_B", "dataset_b_q.npy", "dataset_b_r.npy")

# construir el indice para linear scan para el nuevo dataset
flann.build_index(experimentoB.dataset_r, algorithm="linear")

# ejecutar el linear scan
t0 = time.time()
nns, dists = flann.nn_index(experimentoB.dataset_q, num_neighbors=1, cores=1)
segundos = time.time() - t0

experimentoB.respuesta_ideal(nns, dists, segundos)

# construir el indice k-means tree
t0 = time.time()
flann.build_index(experimentoB.dataset_r, algorithm="kmeans", branching=5)
segundos_construccion = time.time() - t0
print("construcción kmeans tree={:.2f} segundos".format(segundos_construccion))

# probar con distintos checks y guardar los datos de efectividad vs eficiencia
precisiones2 = []
tiempos2 = []
for checks in (1, 10, 20, 40, 60, 80, 100, 200, 400, 600, 800, 1000, 2000, 4000):
    t0 = time.time()
    nns, dists = flann.nn_index(
        experimentoB.dataset_q, num_neighbors=1, cores=1, checks=checks
    )
    segundos = time.time() - t0

    precision, eficiencia = experimentoB.evaluar_busqueda(
        "kmeans tree 5-branching", "checks={:>4}".format(checks), nns, dists, segundos
    )

    precisiones2.append(precision)
    tiempos2.append(eficiencia)

### Comparar efectividad vs eficiencia lograda por k-means tree en ambos datasets

In [ ]:
plt.plot(
    precisiones,
    tiempos,
    label="kmeans tree-dataset A",
    color="b",
    linestyle="-",
    marker="o",
    markerfacecolor="g",
    markersize=8,
)
plt.plot(
    precisiones2,
    tiempos2,
    label="kmeans tree-dataset B",
    color="r",
    linestyle="-",
    marker="o",
    markerfacecolor="m",
    markersize=8,
)
plt.title(experimentoB.nombre)
plt.xlabel("Precisión NN   [% de LS]")
plt.ylabel("Tiempos   [% de LS]")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.show()

Notar la diferencia de resultados que obtiene el mismo árbol (kmeans tree con `branching=5`) al indexar dos datasets distintos.
Ambos datasets tienen la misma cantidad de vectores y dimensionalidad.

En el dataset A es posible lograr mejores aproximaciones que en el dataset B. Por ejemplo, en el dataset A se puede reducir el tiempo de búsqueda al 4% del linear scan logrando un 95% de respuestas correctas, mientras que en el dataset B al reducir el tiempo de búsqueda a un 5% logra un 86% de respuestas correctas.

La performance de las búsqueda aproximadas (esto es, la precisión lograda cuando se reduce el tiempo de búsqueda) y la performance de los índices en general **depende de cómo están distribudos los vectores en el espacio**. Hablaremos más sobre estos en las siguientes semanas.

# Parte 4: Probar distintos branching del k-means tree

En un mismo dataset (DATASET_B) se probarán varios k-means tree con distintos branching para ver cuál logra mejores aproximaciones (esto es, el que logra reducir más el tiempo de búsqueda logrando mejor calidad de respuesta).

In [ ]:
checks_a_probar = (1, 10, 20, 40, 60, 80, 100, 200, 400, 600, 800, 1000, 2000, 4000)
precisiones1 = []
tiempos1 = []
precisiones2 = []
tiempos2 = []
precisiones3 = []
tiempos3 = []

##################
# construir indice
t0 = time.time()
flann1 = pyflann.FLANN()
flann1.build_index(experimentoB.dataset_r, algorithm="kmeans", branching=5)
segundos_construccion = time.time() - t0
print(
    "construcción kmeans tree 5-branching  = {:.2f} segundos".format(
        segundos_construccion
    )
)

# busquedas aproximadas
for checks in checks_a_probar:
    t0 = time.time()
    nns, dists = flann1.nn_index(
        experimentoB.dataset_q, num_neighbors=1, cores=1, checks=checks
    )
    segundos = time.time() - t0
    precision, eficiencia = experimentoB.evaluar_busqueda(
        "kmeans tree 5-branching  ", "checks={:>4}".format(checks), nns, dists, segundos
    )
    precisiones1.append(precision)
    tiempos1.append(eficiencia)

##################
# construir indice
t0 = time.time()
flann2 = pyflann.FLANN()
flann2.build_index(experimentoB.dataset_r, algorithm="kmeans", branching=20)
segundos_construccion = time.time() - t0
print(
    "construcción kmeans tree 20-branching = {:.2f} segundos".format(
        segundos_construccion
    )
)

# busquedas aproximadas
for checks in checks_a_probar:
    t0 = time.time()
    nns, dists = flann2.nn_index(
        experimentoB.dataset_q, num_neighbors=1, cores=1, checks=checks
    )
    segundos = time.time() - t0
    precision, eficiencia = experimentoB.evaluar_busqueda(
        "kmeans tree 20-branching ", "checks={:>4}".format(checks), nns, dists, segundos
    )
    precisiones2.append(precision)
    tiempos2.append(eficiencia)

##################
# construir indice
t0 = time.time()
flann3 = pyflann.FLANN()
flann3.build_index(experimentoB.dataset_r, algorithm="kmeans", branching=200)
segundos_construccion = time.time() - t0
print(
    "construcción kmeans tree 200-branching= {:.2f} segundos".format(
        segundos_construccion
    )
)

# busquedas aproximadas
for checks in checks_a_probar:
    t0 = time.time()
    nns, dists = flann3.nn_index(
        experimentoB.dataset_q, num_neighbors=1, cores=1, checks=checks
    )
    segundos = time.time() - t0
    precision, eficiencia = experimentoB.evaluar_busqueda(
        "kmeans tree 200-branching", "checks={:>4}".format(checks), nns, dists, segundos
    )
    precisiones3.append(precision)
    tiempos3.append(eficiencia)

### Graficar comparación

In [ ]:
plt.plot(
    precisiones1,
    tiempos1,
    label="kmeans tree 5-b  ",
    color="b",
    linestyle="-",
    marker="o",
    markerfacecolor="g",
    markersize=8,
)
plt.plot(
    precisiones2,
    tiempos2,
    label="kmeans tree 20-b ",
    color="r",
    linestyle="-",
    marker="o",
    markerfacecolor="m",
    markersize=8,
)
plt.plot(
    precisiones3,
    tiempos3,
    label="kmeans tree 200-b",
    color="g",
    linestyle="-",
    marker="o",
    markerfacecolor="y",
    markersize=8,
)
plt.title(experimentoB.nombre)
plt.xlabel("Precisión NN   [% de LS]")
plt.ylabel("Tiempos   [% de LS]")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.plot([0.95, 0.95], [0.58, 0], linewidth=1, linestyle="-", marker=" ", color="r")
plt.text(0.9, 0.6, "95% precisión", ha="center", fontsize=9, color="red")
plt.show()

Si el branching es muy bajo el árbol tendrá mucha altura (porque tendrá que hacer muchas subdivisiones), y si el branching es muy grande el árbol quedará muy ancho (cada nodo tendrá demasiados hijos). Es necesario encontrar un buen balance para tener un árbol eficiente para buscar (esto depende del número de vectores y también de cómo están distribuidos los vectores).

Un punto de comparación interesante es el **tiempo requerido para lograr un 95% de precisión** de la búsqueda aproximada. Según el experimento anterior (en mi computador, puede variar), para cada árbol estos son los puntos más cercanos al 95% de precisión:

```
kmeans tree 5-branching  :	checks=400	precision=93.6%	segundos=3.15	(9.8% del tiempo de Linear Scan)
kmeans tree 5-branching  :	checks=600	precision=96.2%	segundos=4.80	(14.9% del tiempo de Linear Scan)

kmeans tree 20-branching :	checks=600	precision=93.2%	segundos=1.49	(4.6% del tiempo de Linear Scan)
kmeans tree 20-branching :	checks=800	precision=95.4%	segundos=2.03	(6.3% del tiempo de Linear Scan)

kmeans tree 200-branching:	checks=600	precision=94.1%	segundos=1.83	(5.7% del tiempo de Linear Scan)
kmeans tree 200-branching:	checks=800	precision=96.3%	segundos=2.26	(7.0% del tiempo de Linear Scan)
```

Según estos datos, k-means tree con `branching=20` alcanza un 95% de precisión en el menor tiempo, mientras que `branching=5` resulta ser un valor muy pequeño (la búsqueda es más lenta), y con `branching=200` resulta ser muy grande (la búsqueda es levemente más lenta).
